In [ ]:
!pip install llama-cpp-python huggingface_hub

# Downloaden der GGUF Modelle
Um zum ersten Mal ein LLM bei uns auszuführen, wollen wir zunächst ein gguf Modelle verwenden. Für den Anfang verwenden wir das Llama-2-7B-GGUF und das Mistral-7B-Instruct-v0.1-GGUF Modell. Diese Modelle sind relativ klein und können direkt auf einer CPU ausgeführt werden. Um diser Modelle zu verwenden, müssen wir diese zunächst herunterladen. Dafür verwenden wir die Funktion hf_hub_download aus dem huggingface_hub Modul um die Modelle von huggingface herunterzuladen. Weitere gguf Modelle können zum Beispiel  [hier](https://huggingface.co/TheBloke) gefunden werden.

In [ ]:
from huggingface_hub import hf_hub_download

llama_path = hf_hub_download(repo_id="TheBloke/Llama-2-7B-GGUF", filename="llama-2-7b.Q4_0.gguf")
mistral_path = hf_hub_download(repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
                               filename="mistral-7b-instruct-v0.1.Q4_0.gguf")

## Ausführen der gguf Modelle
Nach dem herunterladen der Modelle, können wir diese nun in unserem Python Code verwenden. Dafür verwenden wir die llama-cpp-python Bibliothek, die uns eine einfache Schnittstelle zu den Modellen bietet. Um ein Modell zu verwenden, müssen wir zunächst eine Instanz der Llama Klasse erstellen und den Pfad zum Modell übergeben. Anschließend können wir die Instanz wie eine Funktion aufrufen und die Eingabe (Prompt) übergeben. Die Ausgabe ist ein Python Dictionary, das die Antwort des Modells enthält.

In [ ]:
from llama_cpp import Llama

llama = Llama(llama_path)
response = llama("What is the capital of France?")
print(response)

Die Ausgabe ist eine Python Map. Wenn wir diese etwas schöner formattieren könnte die Ausgabe wie folgt aussehen.

```json
{
  "id": "cmpl-97de95d4-101f-4193-a544-65d26211ddba",
  "object": "text_completion",
  "created": 1707901428,
  "model": "/home/jt/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-GGUF/snapshots/b4e04e128f421c93a5f1e34ac4d7ca9b0af47b80/llama-2-7b.Q4_0.gguf",
  "choices": [
    {
      "text": "1. hopefully you got a good mark in your Geography lesson today!\n",
      "index": 0,
      "logprobs": "None",
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 13,
    "completion_tokens": 16,
    "total_tokens": 29
  }
}

## Zugriff auf die Antwort

In [1]:
print(response["choices"][0]["text"])

NameError: name 'response' is not defined

## Model mit GPU Unterstützung
Aktuell ist unsere Modell noch etwas langsam um dies nun zu beschleunigen wollen wir eine GPU nutzen. Um unser Model auf der GPU auszuführen, müssen wir zunächst den Laufzeit-Typ auf GPU umstellen. Dafür klicken wir in der Toolbar unter Laufzeit auf Laufzeittyp ändern und wählen nun die T4 GPU aus. Anschließend installieren wir llama-cpp-python erneut, diesmal mit der Option -DLLAMA_CUBLAS=on um CUDA zu aktivieren.

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir

In [ ]:
llama = Llama(
    llama_path,
    n_gpu_layers=-1, # Verschiebt die Berechnung auf die GPU
    verbose=False, # Entfernt die Logausgaben
)

In [ ]:
response = llama("What is the capital of France?")
print(response["choices"][0]["text"])

## Bessere Antworten bekommen
Das Llama Model was wir verwenden ist ein sogenanntes Foundation Model. Diese Modelle sind meißt nicht direkt in der Anwendung nutzbar, sondern müssen noch finegetuned werden. Ein soein gefinetuntes Modell ist das Mistral-7B-Instruct-v0.1-GGUF Modell. Dieses Modell ist speziell darauf trainiert worden, Anweisungen zu befolgen. Dieses Modell haben wir bereits heruntergeladen und können es nun verwenden. 

In [ ]:
mistral = Llama(
    mistral_path,
    n_gpu_layers=-1, 
    verbose=False,
)

In [ ]:
response = mistral("What is the capital of France?")
print(response["choices"][0]["text"])

In [ ]:
response = mistral(
    "Write me a poem about the technology of the future.",
    max_tokens=512,
)
print(response["choices"][0]["text"])

# Chatbot erstellung

In [ ]:
current_messages = [
    {
        "role": "system",
        "content": "answer in the style of a pirate",
    }
]

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    
    current_messages.append({
        "role": "user",
        "content": user_input,
    })
    
    response = mistral.create_chat_completion(current_messages, max_tokens=512)
    current_messages.append(response["choices"][0]["message"])
    print(response["choices"][0]["message"]["content"])

## Streaming der Antworten

In [ ]:
response = mistral(
    "Write me a poem about the technology of the future.",
    max_tokens=512,
    stream=True,
)

for message in response:
    print(message["choices"][0]["text"], end="")

## Aufgabe
baut das Streaming in unseren Chatbot ein

In [ ]:
current_messages = [
    {
        "role": "system",
        "content": "answer in the style of a pirate",
    }
]

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    
    current_messages.append({
        "role": "user",
        "content": user_input,
    })
    
    response = mistral.create_chat_completion(
        current_messages, 
        max_tokens=512,
        stream=True,
    )
    
    message = ""
    for chunk in response:
        part = chunk["choices"][0]["message"]["content"]
        message += part
        print(part, end="")
    
    current_messages.append({
        "role": "assistant",
        "content": message,
    })

# Exllamav2
Wir wollen uns nun eine weiter Quantisierung von LLM Modellen anschauen. Dafür verwenden wir das Exllamav2, welches exl2 Modelle verwendet.

Das besondere an Exllamav2 ist, ist das ist sehr schnell ist, weshalb wir es uns nun im folgenden anschauen wollen. Zunächst müssten wir hierfür exllamav2 installieren und ein Modell herunterladen. Viele Modelle können zum Beispiel [hier](https://huggingface.co/LoneStriker) gefunden werden.

In [2]:
!pip install exllamav2

In [ ]:
from huggingface_hub import snapshot_download
wizard_math_path = snapshot_download(repo_id="LoneStriker/WizardMath-7B-V1.1-4.0bpw-h6-exl2", revision="main")

In [5]:
from exllamav2 import(
    ExLlamaV2,
    ExLlamaV2Config,
    ExLlamaV2Cache,
    ExLlamaV2Tokenizer,
)

from exllamav2.generator import (
    ExLlamaV2Sampler, ExLlamaV2StreamingGenerator
)

# Erstellung des Modells
config = ExLlamaV2Config() # Einstellungen für das Modell zum Beispiel welches Modell verwendet werden soll oder die maximale Sequenz Länge des LLMs
config.model_dir = wizard_math_path
config.prepare()
model = ExLlamaV2(config) 

cache = ExLlamaV2Cache(model, lazy=True)
model.load_autosplit(cache)

# Erstellung des Tokenizers
tokenizer = ExLlamaV2Tokenizer(config)
generator = ExLlamaV2StreamingGenerator(model, cache, tokenizer)
generator.set_stop_conditions([tokenizer.eos_token_id])

# Einstellungen für die Anfrage
settings = ExLlamaV2Sampler.Settings() # Einstellungen für die Anfrage zum Beispiel die Temperatur oder die maximale Anzahl an Tokens

ImportError: /home/jt/.cache/torch_extensions/py311_cu121/exllamav2_ext/exllamav2_ext.so: undefined symbol: _ZN3c107WarningC1ESt7variantIJNS0_11UserWarningENS0_18DeprecationWarningEEERKNS_14SourceLocationESsb

In [ ]:
instruction = """
Meredith is a freelance blogger who writes about health topics and submits to clients each
day as her permanent job. A blog article takes an average of 4 hours to research and write about. Last week,
she wrote 5 articles on Monday and 2/5 times more articles on Tuesday than on Monday. On Wednesday,
she wrote twice the number of articles she wrote on Tuesday. Calculate the total number of hours she spent
writing articles in the three days.
"""
prompt = f"""
Below is an instruction that describes a task. Write a
response that appropriately completes the request.\n\n###
Instruction:\n{instruction}\n\n### Response:
"""

# Umwandlung der Anfrage in Tokens
instruction_ids = tokenizer.encode(prompt, add_bos=True) # add bos token sorgt dafür, dass der start token hinzugefügt wird
generator.begin_stream(instruction_ids, settings)

while True:
    chunk, eos, _ = generator.stream()
    if eos:
        break
    print(chunk, end="")